# Australian Geoscience Datacube API
Perform band maths and produce a Normalised Difference Vegetation Index (NDVI) file.

In [ ]:
# do this if you want to import python from a specific agdc-v2 code directory

import sys

def setup_env(agdc2dev):

    paths=sys.path
    #paths.append(agdc2dev)
    paths.insert(0, agdc2dev)  #prepend the agdc-v2 path for import
    
    print "Now, your import search paths: "
    for p in  sys.path:
        print p
        
        


In [ ]:
# Please change to your own agdc-v2 code dir in below function call:
my_agdc2 = '/g/data1/u46/fxz547/Githubz/agdc-v2'
setup_env( my_agdc2 )

In [ ]:

%matplotlib inline
from matplotlib import pyplot as plt

import datacube.api
from pprint import pprint

from IPython.display import display
from collections import defaultdict

import xarray as xr
import xarray.ufuncs

from datacube.api import API
from datacube.index import index_connect
from datacube.config import LocalConfig
from datacube.api._conversion import to_datetime
from datacube.api import make_mask, describe_flags


In [ ]:
# which datacube connection
# it's up to you to use this hack

force_prod = True 

if force_prod:
    prod_config = LocalConfig.find(['/g/data/v10/public/modules/agdc-py2-prod/1.0.2/datacube.conf'])
    prod_index = index_connect(prod_config, application_name='api-WOfS-dev')
    dc = API(prod_index)
else:
    dc = API(application_name='api-WOfS-dev')

In [ ]:
nbar = dc.get_dataset(product='nbar', platform='LANDSAT_5', 
                      y=(-34.95,-35.05), x=(148.95,149.05), 
                      variables=['band_3', 'band_4'])

We are working on a semantics layer to be able to use alias for band names, but for now, we have to rely on the knowledge that band_30 is red, and band_40 is near-infrared.

In [ ]:
red = nbar.band_3
nir = nbar.band_4

We can select the first time index and plot the first timeslice.

In [ ]:
nir.isel(time=0).plot()

We can also select a range on the spatial dimensions.

In [ ]:
red.isel(time=0).sel(y=slice(-3920000.5,-3926000.5), x=slice(1542112, 1563962)).plot()

NDVI compares the red and near-infrared bands:

In [ ]:
ndvi = (nir - red) / (nir + red)
ndvi.name = 'Normalised Difference Vegetation Index'

The `ndvi` array has values across the spatial and time dimensions.

In [ ]:
ndvi

We can select an individual time layer to make a labelled plot:

In [ ]:
ndvi.isel(time=0).plot()

We have set up the calculation of NDVI to happen for all timeslices. It will be computed when the data is requested, either explicitly by calling `ndvi.load()`, or by a function that accesses the data, such as plotting:
(This can take a while...)

In [ ]:
ndvi.isel(time=[3,4,9,13]).plot(col='time')

The xarray can be turned into a Dataset, which can then be saved across all of the timeseries to a NetCDF file.

In [ ]:
ds_ndvi = ndvi[:100,:100,:100].to_dataset(name='ndvi')
ds_ndvi.to_netcdf('ndvi.nc')

### Transposing dimensions

In [ ]:
visible = dc.get_data_array(product='nbar', platform='LANDSAT_5', 
                      y=(-35.26,-35.32), x=(149.09,149.17), 
                      variables=['band_1', 'band_2', 'band_3'])
visible.dims

We will clip and scale the image to improve the contrast of the visible bands.

In [ ]:
fake_saturation = 1500
clipped_visible = visible.where(visible<fake_saturation).fillna(fake_saturation)
max_val = clipped_visible.max(['y', 'x'])
scaled = (clipped_visible / max_val)

In [ ]:
rgb = scaled.transpose('time', 'y', 'x', 'variable')
rgb.dims

In [ ]:
plt.imshow(rgb.isel(time=16))

In [ ]:
import matplotlib.image
matplotlib.image.imsave('ndvi.png', rgb.isel(time=16))

## Behind the scenes

The ndvi result is performed by chaining a series of operations together on a per-chunk basis. The execution tree, including the masking on the `nodata` value done by the API, ends up being the same for each chunk. The graph can be read from the bottom up, with the ouput array chunks at the top.
(Double-click the tree-graph image below to zoom)

In [ ]:
ndvi.data.visualize()

If we look at a single chunk, the NDVI calculation can be seen where the lines cross over to the `add` and `sub` circles.
Some optimizarion has taken place: the `div` operation has been combined with another inline function, and the other chunks have been discarded.

In [ ]:
partial = ndvi[0,0,0]
partial.data.visualize(optimize_graph=True)